In [314]:
import pandas as pd,numpy

In [315]:
df = pd.read_csv("all_seasons.csv") 
df1 = pd.read_csv("injuries_2010-2020.csv") 

In [316]:
df1['date1']=pd.to_datetime(df1['Date'], format='%Y-%m-%d') #Converting Date column to Datetimetype
df1=df1.drop(columns=['Date'])                              #Dropping original columns
df1['year'] = pd.DatetimeIndex(df1['date1']).year           #Seperating year from date
df1= df1.rename(columns={"year": "Season"})
df['Season'] = df['season'].str[:4]                         #Renaming year as Season
df=df.astype({"Season": int })                               
df_filter1 = df[['player_name','player_height','player_weight','age','Season']]     #filtering out necessary parameters
df_filter1 = df_filter1.rename(columns={"player_name": "player_name"})        
#df1_filter1 = df1.rename(columns={"Relinquished": "player_name_r"})
#df1_filter2 = df1_filter1[df1_filter1['player_name_r'].notna()]

In [317]:
df1["player_name"] = df1["Acquired"].combine_first(df1["Relinquished"]) #constructing player name from combining acquired and Relinquished

In [318]:
merge1 = pd.merge(df1,df_filter1,on=['player_name','Season'])   #merging the injury and player Bio based on player_name and Season

In [319]:
merge1= merge1[~(merge1['Notes'] == "placed on IL")]        #removing the entries which contain placed on IL

In [320]:
merge1=merge1.drop(['Acquired', 'Relinquished'], axis=1)   #dropping acquired and relinquished since we already obtained player name by combining them

In [321]:
merge1 = merge1.sort_values(by=['player_name','date1'])    #sorting the data by player name and date

In [322]:
merge1 = merge1[~merge1['Notes'].str.contains("activat")]  #removing rows containing "Activated" since those are not injuries

In [323]:
merge1= merge1[~(merge1['Notes'] == "placed on IL")]           
merge1= merge1[~(merge1['Notes'] == "returned to lineup")]  #removing the entries which say returned to lineup


In [324]:
searchfor = ['flu','illness','COVID']       #removing some entries which contain specific words as those cannot be classified as injuries
merge1 = merge1[~merge1.Notes.str.contains('|'.join(searchfor))]

In [325]:
merge1['Notes'] = merge1['Notes'].str.replace(r'(DNP)','').replace(r'(DTD)','')  #removing (DNP) and (DTD) from entries to bring similarity between string
merge1['Notes'] =merge1['Notes'].str.replace(r" \(.*\)","")        #removing unnecessary special characters


In [326]:
merge1['Previous_entry'] = merge1.groupby('player_name')['Notes'].shift(1)      #shifting player name
merge1['Previous_date'] = merge1.groupby('player_name')['date1'].shift(1)       #shifting date of previous row for comparison purposes

In [327]:
merge1['Difference'] = merge1['date1'].sub(merge1['Previous_date'], axis=0)  #creating a difference column to get the difference in days between two entries

In [328]:
merge1['Difference']=merge1['Difference'].fillna(pd.Timedelta(seconds=0))   #obtaining the difference in days


In [329]:
merge1['Difference'] = merge1['Difference'].dt.days.astype('int16')   #converting difference to integer for easier comparison

In [330]:
merge1['Deletion_status']= (merge1.Previous_entry.isin(merge1.Notes)) & (merge1['Difference']<=5)  #delete if Previous entry contains current entry and time difference is less than 5 days(refers to the same injury)          
merge1 = merge1[merge1.Deletion_status != True] 

In [331]:
#rearranging the data to make it presentable

merge1.drop(['Deletion_status','Previous_entry','Previous_date','Difference'],axis=1,inplace=True) 
merge1=merge1.reset_index(drop=True)
merge1=merge1.rename(columns={"date1": "Date"})
merge1=merge1.rename(columns={"player_name": "Player_Name"})
merge1 =merge1[['Team','Date','Player_Name', 'Notes','Season','player_height','player_weight','age']]

In [332]:
merge1

,Team,Date,Player_Name,Notes,Season,player_height,player_weight,age
0,Rockets,2010-11-07,Aaron Brooks,placed on IL with sprained left ankle,2010,182.88,73.028312,26.0
1,Kings,2012-12-30,Aaron Brooks,sore left ankle,2012,182.88,73.028312,28.0
2,Rockets,2014-01-15,Aaron Brooks,tendinitis in knee,2014,182.88,73.028312,30.0
3,Bulls,2015-11-19,Aaron Brooks,strained left hamstring,2015,182.88,73.028312,31.0
4,Bulls,2015-12-05,Aaron Brooks,strained left hamstring,2015,182.88,73.028312,31.0
...,...,...,...,...,...,...,...,...
7013,76ers,2019-04-29,Zhaire Smith,placed on IL,2019,190.50,92.986360,21.0
7014,76ers,2019-12-08,Zhaire Smith,right leg injury,2019,190.50,92.986360,21.0
7015,Pelicans,2019-10-18,Zion Williamson,torn meniscus in right knee,2019,198.12,128.820128,19.0
7016,Pelicans,2020-02-08,Zion Williamson,sprained left ankle,2020,200.66,128.820128,20.0


In [333]:
merge1.to_csv('injury_heavy_filtered.csv', index=False) #converting dataframe to csv